In [1]:
# TODO: node2vec特徴量を作成 → Baselineに追加してみる
# TODO: seen, unseenでユーザーを分けてモデル作成
# TODO: seen:stratifiedKFold, unseen:GroupKFoldでCV
# TODO: seenでしか作れない特徴量エンジニアリング


In [2]:
import polars as pl
from sklearn.preprocessing import LabelEncoder

from src.data import load_data

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(10)
pl.Config.set_tbl_cols(20)


polars.config.Config

In [3]:
train, test, anime = load_data()
display(anime)
display(train)
display(test)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
i16,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""","""TV""","""99""","""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""","""TV""","""26""","""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""","""Manga""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""","""TV""","""10""","""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677
3,"""Comedy, Ecchi, Fantasy, School""","""星刻の竜騎士""","""TV""","""12""","""Apr 5, 2014 to Jun 21, 2014""","""Media Factory, AT-X, Sony Music Communications, Tsukuru no Mori""","""Funimation""","""C-Station""","""Light novel""","""24 min. per ep.""","""R+ - Mild Nudity""",170220,8723,118202,3753,8034,31508
4,"""Comedy, Harem, Romance, Sci-Fi, Shounen, Space""","""天地無用！""","""TV""","""26""","""Apr 2, 1995 to Sep 24, 1995""","""TV Tokyo, Pioneer LDC""","""Funimation, Geneon Entertainment USA""","""AIC""","""Original""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",62599,2565,39890,2093,1986,16065
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1995,"""Adventure, Romance, Mecha, Military, Sci-Fi, Horror, Space, Drama""","""BLUE GENDER""","""TV""","""26""","""Oct 8, 1999 to Mar 31, 2000""","""Youmex, Toshiba EMI""","""Funimation""","""AIC""","""Original""","""23 min. per ep.""","""R+ - Mild Nudity""",65360,2738,31583,2534,3195,25310
1996,"""Ecchi, Romance, School""","""フォトカノ""","""TV""","""13""","""Apr 5, 2013 to Jun 28, 2013""","""TBS, Enterbrain, BS-TBS, RAY""","""Sentai Filmworks""","""Madhouse""","""Visual novel""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",93558,6242,45399,3415,8680,29822
1997,"""Action, Fantasy, Military, Sci-Fi""","""GOD EATER""","""TV""","""13""","""Jul 12, 2015 to Mar 26, 2016""","""Bandai Visual, Lantis, Anime Consortium Japan""","""Aniplex of America""","""ufotable""","""Game""","""23 min. per ep.""","""R - 17+ (violence & profanity)""",426616,35000,242285,16731,31573,101027


user_id,anime_id,score
i16,i16,i8
0,49,2
0,122,10
0,234,1
0,263,8
0,318,9
…,…,…
1996,1814,6
1996,1819,7
1996,1850,7


user_id,anime_id
i16,i16
0,27
0,34
0,152
0,297
0,303
…,…
1997,1908
1997,1910
1997,1915


## animeで与えられているものをとりあえず特徴量として使えるようにする
* animeは全部で2000件で重複はない

In [4]:
anime.height


2000

In [5]:
anime["anime_id"].unique().len()


2000

In [6]:
# TODO:discussionを参考に実装


### japanese_name
* クール違いやOVAなどで日本語名が重複してるアニメがあるもののごく少数
* とりあえず使わない（使うとしたらカウントエンコーディングorラベルエンコーディング？）
* シリーズ物か否かというフラグはあり（japanese_nameが完全に一致でなくてもシリーズ物はあるので、それを考慮しないといけない）

In [7]:
# japanese_name カラムで重複どれだけあるか確認
duplicate_japanese_name_list = (
    anime.group_by("japanese_name").len().filter(pl.col("len") > 1)["japanese_name"].to_list()
)
print(f"重複してるアニメの数: {len(duplicate_japanese_name_list)}")


重複してるアニメの数: 59


In [8]:
# japanese_nameが重複してるアニメを確認
anime.filter(pl.col("japanese_name").is_in(duplicate_japanese_name_list)).sort("japanese_name")


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
i16,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
189,"""Action, Adventure, Comedy, Historical, Samurai, Fantasy, Seinen""","""DRIFTERS""","""TV""","""12""","""Oct 7, 2016 to Dec 23, 2016""","""Hoods Entertainment, NBCUniversal Entertainment Japan, Shounen Gahousha""","""Funimation""","""Hoods Drifters Studio""","""Manga""","""23 min. per ep.""","""R - 17+ (violence & profanity)""",410926,27263,252770,12026,15984,102883
1167,"""Action, Adventure, Comedy, Fantasy, Historical, Samurai, Seinen""","""DRIFTERS""","""OVA""","""2""","""Dec 23, 2017""","""Unknown""","""Unknown""","""Hoods Drifters Studio""","""Manga""","""26 min. per ep.""","""R - 17+ (violence & profanity)""",60741,1462,26092,686,418,32083
50,"""Action, Adventure, Comedy, Magic, Fantasy, Shounen""","""FAIRY TAIL（フェアリーテイル）""","""TV""","""175""","""Oct 12, 2009 to Mar 30, 2013""","""TV Tokyo, Dentsu, Pony Canyon, Kodansha, DAX Production, Sakura Create""","""Funimation""","""Satelight, A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",1312470,185463,737096,108697,148408,132806
1184,"""Action, Adventure, Comedy, Fantasy, Magic, Shounen""","""FAIRY TAIL（フェアリーテイル）""","""TV""","""102""","""Apr 5, 2014 to Mar 26, 2016""","""TV Tokyo, Dentsu""","""Funimation""","""A-1 Pictures, Bridge""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",716382,101344,440657,43729,50700,79952
364,"""Action, Adventure, Super Power, Fantasy, Shounen""","""HUNTER×HUNTER（ハンター×ハンター）""","""TV""","""62""","""Oct 16, 1999 to Mar 31, 2001""","""Fuji TV""","""VIZ Media""","""Nippon Animation""","""Manga""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",416227,28061,260968,15535,15100,96563
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
161,"""Ecchi, School, Shounen""","""食戟のソーマ 弍ノ皿""","""OVA""","""2""","""May 1, 2017 to Jul 4, 2017""","""Warner Bros. Japan, Shueisha""","""Unknown""","""J.C.Staff""","""Manga""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",120474,4234,88704,1314,619,25603
1390,"""Ecchi, School, Shounen""","""食戟のソーマ 弍ノ皿""","""TV""","""13""","""Jul 2, 2016 to Sep 24, 2016""","""Dentsu, Lantis, Mainichi Broadcasting System, KlockWorx, Showgate, Shueisha""","""Sentai Filmworks""","""J.C.Staff""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",802974,37811,671406,9753,7700,76304
198,"""Comedy, School, Shounen, Sports""","""黒子のバスケ""","""TV""","""25""","""Apr 8, 2012 to Sep 22, 2012""","""Bandai Visual, Lantis, Nihon Ad Systems, Mainichi Broadcasting System, Banpresto, Shueisha, BS11""","""Unknown""","""Production I.G""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",810207,44424,609160,23011,21160,112452


### type
* 7種類の値しかない → label encodeする

In [9]:
# typeの値を
anime["type"].unique()


type
str
"""Unknown"""
"""Movie"""
"""ONA"""
"""Special"""
"""OVA"""
"""TV"""
"""Music"""


In [10]:
# typeの値がそれぞれいくつあるか確認
anime.group_by("type").len().sort("len", descending=True)


type,len
str,u32
"""TV""",1436
"""Movie""",272
"""OVA""",142
"""Special""",97
"""ONA""",50
"""Unknown""",2
"""Music""",1


In [11]:
# typeの値をlabel encodeする
le = LabelEncoder()
anime = anime.with_columns(pl.Series("type", le.fit_transform(anime["type"])))
anime.head()


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
i16,str,str,i64,str,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,"""99""","""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,"""26""","""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""","""Manga""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,"""10""","""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677
3,"""Comedy, Ecchi, Fantasy, School""","""星刻の竜騎士""",5,"""12""","""Apr 5, 2014 to Jun 21, 2014""","""Media Factory, AT-X, Sony Music Communications, Tsukuru no Mori""","""Funimation""","""C-Station""","""Light novel""","""24 min. per ep.""","""R+ - Mild Nudity""",170220,8723,118202,3753,8034,31508
4,"""Comedy, Harem, Romance, Sci-Fi, Shounen, Space""","""天地無用！""",5,"""26""","""Apr 2, 1995 to Sep 24, 1995""","""TV Tokyo, Pioneer LDC""","""Funimation, Geneon Entertainment USA""","""AIC""","""Original""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",62599,2565,39890,2093,1986,16065


### episodes
* それ以外は数値として扱う
* Unknownは欠損として扱う


In [12]:
# episodesのユニーク確認 --> 'unknown'がある
anime["episodes"].unique().to_list()


['27',
 '1',
 '52',
 '73',
 'Unknown',
 '21',
 '148',
 '69',
 '77',
 '8',
 '101',
 '500',
 '366',
 '112',
 '5',
 '14',
 '37',
 '2',
 '167',
 '131',
 '146',
 '26',
 '224',
 '76',
 '9',
 '75',
 '203',
 '61',
 '40',
 '7',
 '49',
 '6',
 '70',
 '276',
 '23',
 '51',
 '12',
 '43',
 '191',
 '102',
 '18',
 '48',
 '16',
 '109',
 '36',
 '25',
 '13',
 '54',
 '46',
 '94',
 '19',
 '15',
 '64',
 '180',
 '161',
 '45',
 '97',
 '22',
 '47',
 '3',
 '74',
 '39',
 '145',
 '20',
 '50',
 '93',
 '154',
 '128',
 '100',
 '110',
 '220',
 '62',
 '153',
 '38',
 '78',
 '60',
 '175',
 '170',
 '11',
 '10',
 '178',
 '127',
 '84',
 '120',
 '103',
 '150',
 '34',
 '24',
 '192',
 '291',
 '99',
 '201',
 '147',
 '4',
 '114']

In [13]:
# 'unknown'があるアニメを確認 --> 18個
anime.filter(pl.col("episodes") == "Unknown").height


18

In [14]:
# unknownを-1にする
anime = anime.with_columns(pl.col("episodes").str.replace("Unknown", "-1"))
# カラムを数値型にする
anime = anime.with_columns(pl.col("episodes").cast(pl.Int32))
# -1を欠損値に置き換える
anime = anime.with_columns(pl.when(pl.col("episodes") == -1).then(None).otherwise(pl.col("episodes")).alias("episodes"))

anime.head()


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
i16,str,str,i64,i32,str,str,str,str,str,str,str,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""","""Manga""","""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""","""Manga""","""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677
3,"""Comedy, Ecchi, Fantasy, School""","""星刻の竜騎士""",5,12,"""Apr 5, 2014 to Jun 21, 2014""","""Media Factory, AT-X, Sony Music Communications, Tsukuru no Mori""","""Funimation""","""C-Station""","""Light novel""","""24 min. per ep.""","""R+ - Mild Nudity""",170220,8723,118202,3753,8034,31508
4,"""Comedy, Harem, Romance, Sci-Fi, Shounen, Space""","""天地無用！""",5,26,"""Apr 2, 1995 to Sep 24, 1995""","""TV Tokyo, Pioneer LDC""","""Funimation, Geneon Entertainment USA""","""AIC""","""Original""","""23 min. per ep.""","""PG-13 - Teens 13 or older""",62599,2565,39890,2093,1986,16065


### aired（下記方針はめんどくさすぎかつそんなに効果ないやろうから保留）
* start_period, end_periodでカラム作る
* 年月日でそれぞれカラム作る
* toがないやつはstart, end同じにする
* to ?はendを欠損にする

In [15]:
# airedのユニーク確認
anime["aired"].unique().to_list()


['Dec 12, 2009',
 'Nov 18, 1995',
 'Apr 8, 2008 to Jul 1, 2008',
 'Jan 12, 2020 to Mar 29, 2020',
 'Jan 22, 2020',
 'Apr 6, 2017',
 'Apr 11, 2002 to Sep 26, 2002',
 'Oct 4, 2018 to Dec 20, 2018',
 'Nov 18, 2016',
 'Oct 4, 2000 to Jan 10, 2001',
 'Apr 8, 2017 to Sep 16, 2017',
 'Sep 19, 2015',
 'May 29, 2010',
 'Jan 4, 2017 to Mar 22, 2017',
 'Oct 6, 2020 to Dec 22, 2020',
 'Jan 11, 2019 to Mar 29, 2019',
 'Jul 5, 2010 to Sep 20, 2010',
 'Jul 7, 2001',
 'Apr 17, 2003 to Sep 25, 2003',
 'Aug 12, 2013',
 'Apr 5, 2012 to Mar 28, 2013',
 'Jan 7, 2010 to Apr 1, 2010',
 'Jul 4, 2020 to Sep 26, 2020',
 'Jul 17, 2004',
 'Oct 6, 2013',
 'Jul 30, 2010 to Nov 17, 2010',
 'Aug 22, 2020',
 'Apr 6, 2015 to Mar 28, 2016',
 'Jul 11, 2004 to Sep 26, 2004',
 'Jan 10, 2020 to Mar 27, 2020',
 'Jul 30, 2011',
 'Jan 13, 2018 to Mar 31, 2018',
 'Jul 9, 2018 to Oct 8, 2018',
 'Sep 29, 2012 to Mar 23, 2013',
 'Jul 6, 2014 to Sep 28, 2014',
 'Jul 27, 2011 to Dec 26, 2012',
 'Dec 31, 2011',
 'Jul 1, 2005 to Sep 1

In [16]:
# toがあるairedを確認
set(anime.filter(pl.col("aired").str.contains("to"))["aired"].to_list())


{'May 22, 2009 to May 26, 2010',
 'Oct 11, 2018 to Jun 27, 2019',
 'Jan 17, 2002 to Apr 17, 2002',
 'Apr 4, 2015 to Jul 18, 2015',
 'Apr 5, 2008 to Sep 25, 2008',
 'Jan 12, 2017 to Mar 16, 2017',
 'Nov 3, 2005 to May 11, 2006',
 'Oct 4, 2010 to Dec 20, 2010',
 'Jan 4, 2017 to Mar 29, 2017',
 'Oct 6, 2020 to Dec 22, 2020',
 'Feb 26, 1986 to Apr 12, 1989',
 'Dec 24, 2015 to May 27, 2016',
 'Apr 7, 2012 to Sep 22, 2012',
 'Jul 30, 2003 to Oct 22, 2003',
 'Jan 6, 2021 to ?',
 'Oct 13, 2019 to Dec 29, 2019',
 'Jul 8, 2019 to Sep 23, 2019',
 'Apr 4, 2010 to Jun 20, 2010',
 'Jul 5, 2016 to Sep 20, 2016',
 'Oct 6, 2017 to Dec 22, 2017',
 'Oct 7, 2007 to Mar 23, 2008',
 'Oct 5, 2007 to Dec 21, 2007',
 'Apr 8, 2017 to Sep 16, 2017',
 'Oct 4, 2017 to Dec 27, 2017',
 'Apr 13, 2005 to Sep 21, 2005',
 'Apr 1, 2017 to Jun 17, 2017',
 'May 3, 2007 to May 4, 2007',
 'Jun 24, 2009 to Apr 21, 2010',
 'Jan 14, 2017 to Mar 25, 2017',
 'Oct 5, 2018 to Dec 21, 2018',
 'Oct 4, 2011 to Dec 27, 2011',
 'Mar 21,

In [17]:
# toがないairedを確認
set(anime.filter(~pl.col("aired").str.contains("to"))["aired"].to_list())


{'2021',
 'Apr 12, 2015',
 'Apr 16, 1988',
 'Apr 16, 2009',
 'Apr 17, 2004',
 'Apr 18, 2003',
 'Apr 18, 2015',
 'Apr 19, 2002',
 'Apr 2, 2010',
 'Apr 20, 2013',
 'Apr 20, 2018',
 'Apr 21, 2005',
 'Apr 21, 2012',
 'Apr 21, 2018',
 'Apr 22, 2009',
 'Apr 23, 2016',
 'Apr 24, 2010',
 'Apr 25, 2015',
 'Apr 26, 2011',
 'Apr 26, 2014',
 'Apr 30, 2009',
 'Apr 4, 2017',
 'Apr 6, 2017',
 'Apr 7, 2017',
 'Apr, 2010',
 'Aug 1, 2009',
 'Aug 12, 2013',
 'Aug 13, 2004',
 'Aug 14, 2009',
 'Aug 15, 2020',
 'Aug 16, 2008',
 'Aug 16, 2013',
 'Aug 16, 2014',
 'Aug 16, 2020',
 'Aug 18, 2012',
 'Aug 18, 2013',
 'Aug 18, 2017',
 'Aug 18, 2018',
 'Aug 19, 2016',
 'Aug 2, 1986',
 'Aug 2, 2007',
 'Aug 2, 2008',
 'Aug 2, 2012',
 'Aug 20, 2013',
 'Aug 20, 2014',
 'Aug 21, 1999',
 'Aug 21, 2004',
 'Aug 21, 2010',
 'Aug 22, 2015',
 'Aug 22, 2020',
 'Aug 25, 2000',
 'Aug 26, 2016',
 'Aug 26, 2017',
 'Aug 3, 2018',
 'Aug 30, 2014',
 'Aug 30, 2019',
 'Aug 31, 2013',
 'Aug 4, 2007',
 'Aug 4, 2017',
 'Aug 4, 2018',
 'Au

### source
* 13カテゴリ → label encodingする

In [18]:
anime["source"].unique()


source
str
"""4-koma manga"""
"""Music"""
"""Game"""
"""Digital manga"""
"""Card game"""
…
"""Novel"""
"""Book"""
"""Visual novel"""


In [19]:
# label encoding
le = LabelEncoder()
anime = anime.with_columns(pl.Series("source", le.fit_transform(anime["source"])))
anime.head()


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch
i16,str,str,i64,i32,str,str,str,str,i64,str,str,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""",6,"""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677
3,"""Comedy, Ecchi, Fantasy, School""","""星刻の竜騎士""",5,12,"""Apr 5, 2014 to Jun 21, 2014""","""Media Factory, AT-X, Sony Music Communications, Tsukuru no Mori""","""Funimation""","""C-Station""",5,"""24 min. per ep.""","""R+ - Mild Nudity""",170220,8723,118202,3753,8034,31508
4,"""Comedy, Harem, Romance, Sci-Fi, Shounen, Space""","""天地無用！""",5,26,"""Apr 2, 1995 to Sep 24, 1995""","""TV Tokyo, Pioneer LDC""","""Funimation, Geneon Entertainment USA""","""AIC""",9,"""23 min. per ep.""","""PG-13 - Teens 13 or older""",62599,2565,39890,2093,1986,16065


### duration 
* 1話ごと、全話分のdurationを分で換算して数値として扱う

In [20]:
# 正規表現を使って時間と分を抽出し、数値に変換
anime = anime.with_columns(
    (
        pl.col("duration").str.extract(r"(\d+) hr", 1).cast(pl.Int32, strict=False).fill_null(0) * 60
        + pl.col("duration").str.extract(r"(\d+) min", 1).cast(pl.Int32, strict=False).fill_null(0)
    ).alias("duration_minutes_per_ep")
)

anime.head(3)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch,duration_minutes_per_ep
i16,str,str,i64,i32,str,str,str,str,i64,str,str,i32,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685,24
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""",6,"""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080,25
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677,24


In [21]:
# 60分以上のものも正常に変換できている
anime.filter(pl.col("duration_minutes_per_ep") > 60).head(3)  # noqa


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch,duration_minutes_per_ep
i16,str,str,i64,i32,str,str,str,str,i64,str,str,i32,i32,i32,i32,i32,i32,i32
23,"""Action, Cars, Sports, Drama, Romance, Seinen""","""頭文字〈イニシャル〉D THIRD STAGE""",0,1,"""Jan 13, 2001""","""OB Planning""","""Funimation""","""Studio Deen""",6,"""1 hr. 45 min.""","""PG-13 - Teens 13 or older""",113842,1340,98309,519,348,13326,105
35,"""Action, Adventure, Comedy, Historical, Demons, Supernatural, Drama, Magic, Romance, Fantasy, Shounen""","""犬夜叉 時代を越える想い""",0,1,"""Dec 22, 2001""","""Unknown""","""VIZ Media""","""Kyoto Animation, Sunrise""",6,"""1 hr. 39 min.""","""PG-13 - Teens 13 or older""",81098,785,70124,458,220,9511,99
39,"""Action, Super Power, Supernatural""","""K MISSING KINGS""",0,1,"""Jul 12, 2014""","""Starchild Records, Mainichi Broadcasting System, KlockWorx, Glovision""","""VIZ Media""","""GoHands""",9,"""1 hr. 13 min.""","""PG-13 - Teens 13 or older""",205323,4443,144088,1788,1492,53512,73


In [22]:
# durationがUnknown --> duration_minutesが0なので、それをNoneにする
anime = anime.with_columns(
    pl.when(pl.col("duration_minutes_per_ep") == 0)
    .then(None)
    .otherwise(pl.col("duration_minutes_per_ep"))
    .alias("duration_minutes_per_ep")
)


In [23]:
# Unknown以外でper ep.がついてないものは全てepisodesが1
anime.filter(~(pl.col("duration").str.contains("per ep") | pl.col("duration").str.contains("Unknown"))).filter(
    pl.col("episodes") != 1
)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch,duration_minutes_per_ep
i16,str,str,i64,i32,str,str,str,str,i64,str,str,i32,i32,i32,i32,i32,i32,i32


In [24]:
# duration_minutes_per_ep * episodesで全話分のdurationを算出する
anime = anime.with_columns((pl.col("duration_minutes_per_ep") * pl.col("episodes")).alias("duration_minutes_all_ep"))
anime.head(3)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch,duration_minutes_per_ep,duration_minutes_all_ep
i16,str,str,i64,i32,str,str,str,str,i64,str,str,i32,i32,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",150428,16552,37234,13009,6948,76685,24,2376
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""",6,"""25 min. per ep.""","""PG-13 - Teens 13 or older""",620736,55482,235371,42786,20017,267080,25,650
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""",6,"""24 min. per ep.""","""PG-13 - Teens 13 or older""",226522,12585,113559,6095,2606,91677,24,240


### rating
* 7カテゴリ → label encodingする

In [25]:
anime["rating"].unique()


rating
str
"""Rx - Hentai"""
"""R+ - Mild Nudity"""
"""G - All Ages"""
"""R - 17+ (violence & profanity)"""
"""PG-13 - Teens 13 or older"""
"""Unknown"""
"""PG - Children"""


In [26]:
# ratingの値をlabel encodingする
le = LabelEncoder()
anime = anime.with_columns(pl.Series("rating", le.fit_transform(anime["rating"].fill_null(""))))
anime.head(3)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,duration,rating,members,watching,completed,on_hold,dropped,plan_to_watch,duration_minutes_per_ep,duration_minutes_all_ep
i16,str,str,i64,i32,str,str,str,str,i64,str,i64,i32,i32,i32,i32,i32,i32,i32,i32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""",6,"""24 min. per ep.""",2,150428,16552,37234,13009,6948,76685,24,2376
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""",6,"""25 min. per ep.""",2,620736,55482,235371,42786,20017,267080,25,650
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""",6,"""24 min. per ep.""",2,226522,12585,113559,6095,2606,91677,24,240


### genres, licensors, producers, studios
* 「,」区切りの(1カラムに複数値の入ったデータの)ワンホットエンコーディング（paoさんのディスカッション参照）
* https://www.guruguru.science/competitions/21/discussions/7885664e-2acd-4191-833f-e1b21d34afc4/


In [27]:
multilabel_cols = ["genres", "producers", "licensors", "studios"]
for c in multilabel_cols:
    # カンマで区切って、explodeで展開してからnuniqueでユニーク数を計算
    nunique = anime.select(pl.col(c).str.split(", ").explode().n_unique()).item()
    print(c, nunique)


genres 42
producers 595
licensors 49
studios 193


In [28]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_cols = ["genres", "producers", "licensors", "studios"]
multilabel_dfs = []
n_components = 50
for c in multilabel_cols:
    list_srs = anime.select(pl.col(c).str.split(", ")).to_series().to_list()
    # MultiLabelBinarizerを使うと簡単に変換できるのでオススメです
    mlb = MultiLabelBinarizer()
    ohe_srs = mlb.fit_transform(list_srs)
    if c in {"genres", "licensors"}:
        # ユニーク数が多くないのでOne-hot表現のまま
        col_df = pl.DataFrame(ohe_srs, schema=[f"ohe_{c}_{name}" for name in mlb.classes_])
    else:
        # ユニーク数が多いので、SVDで次元圧縮する
        svd = TruncatedSVD(n_components=n_components)
        svd_arr = svd.fit_transform(ohe_srs)
        col_df = pl.DataFrame(svd_arr, schema=[f"svd_{c}_{ix}" for ix in range(n_components)])
    col_df = col_df.select(pl.all().shrink_dtype())
    multilabel_dfs.append(col_df)

multilabel_df = pl.concat(multilabel_dfs, how="horizontal")
multilabel_df.head(3)


ohe_genres_Action,ohe_genres_Adventure,ohe_genres_Cars,ohe_genres_Comedy,ohe_genres_Dementia,ohe_genres_Demons,ohe_genres_Drama,ohe_genres_Ecchi,ohe_genres_Fantasy,ohe_genres_Game,…,svd_studios_40,svd_studios_41,svd_studios_42,svd_studios_43,svd_studios_44,svd_studios_45,svd_studios_46,svd_studios_47,svd_studios_48,svd_studios_49
i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,1,0,0,0,0,0,0,…,0.00317,0.000122,0.000014,-0.000128,0.000014,-0.000186,0.000248,0.000044,-0.000575,-0.000855
0,1,0,0,0,0,0,0,1,0,…,-0.005001,-0.049869,0.449363,-0.513747,-0.018287,-0.029564,-0.002004,-0.00204,-0.009637,-0.000217
0,1,0,0,0,0,0,0,1,0,…,-0.005001,-0.049869,0.449363,-0.513747,-0.018287,-0.029564,-0.002004,-0.00204,-0.009637,-0.000217


In [29]:
anime = pl.concat([anime, multilabel_df], how="horizontal")
anime.head(3)


anime_id,genres,japanese_name,type,episodes,aired,producers,licensors,studios,source,…,svd_studios_40,svd_studios_41,svd_studios_42,svd_studios_43,svd_studios_44,svd_studios_45,svd_studios_46,svd_studios_47,svd_studios_48,svd_studios_49
i16,str,str,i64,i32,str,str,str,str,i64,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,"""Comedy, Sci-Fi, Seinen, Slice of Life, Space""","""宇宙兄弟""",5,99,"""Apr 1, 2012 to Mar 22, 2014""","""Aniplex, Dentsu, YTV, Trinity Sound""","""Sentai Filmworks""","""A-1 Pictures""",6,…,0.00317,0.000122,0.000014,-0.000128,0.000014,-0.000186,0.000248,0.000044,-0.000575,-0.000855
1,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師""",5,26,"""Oct 23, 2005 to Jun 19, 2006""","""Avex Entertainment, Marvelous, SKY Perfect Well Think, Delfi Sound""","""Funimation""","""Artland""",6,…,-0.005001,-0.049869,0.449363,-0.513747,-0.018287,-0.029564,-0.002004,-0.00204,-0.009637,-0.000217
2,"""Adventure, Slice of Life, Mystery, Historical, Supernatural, Fantasy, Seinen""","""蟲師 続章""",5,10,"""Apr 5, 2014 to Jun 21, 2014""","""Aniplex, Kodansha, Delfi Sound""","""Aniplex of America""","""Artland""",6,…,-0.005001,-0.049869,0.449363,-0.513747,-0.018287,-0.029564,-0.002004,-0.00204,-0.009637,-0.000217


### サブしてみる

In [30]:
import pickle

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold


# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Create a column for the fold number
train = train.with_columns(pl.lit(-1).alias("fold"))

# Assign the fold number to each row
for fold, (_, val_index) in enumerate(skf.split(train, train["user_id"])):
    train[val_index, "fold"] = fold


# Merge the train data with the anime meta data
train_merged = train.join(anime, how="left", on="anime_id")

# Prepare the test data
test_merged = test.join(anime, how="left", on="anime_id")


# Training and evaluation with LightGBM
scores_lgb = []
models_lgb = []

for fold in range(5):
    print(f"Training for fold {fold}...")

    # Prepare the train and validation data
    train_data = train_merged.filter(pl.col("fold") != fold)
    val_data = train_merged.filter(pl.col("fold") == fold)

    # Define the features and the target
    features = [
        "user_id",
        "anime_id",
        # "genres",
        # "japanese_name",
        "type",
        "episodes",
        # "aired",
        # "producers",
        # "licensors",
        # "studios",
        "source",
        # "duration",
        "rating",
        "members",
        "watching",
        "completed",
        "on_hold",
        "dropped",
        "plan_to_watch",
        "duration_minutes_per_ep",
        "duration_minutes_all_ep",
    ]
    features = features + multilabel_df.columns
    target = "score"

    # Prepare the LightGBM datasets
    lgb_train = lgb.Dataset(train_data[features].to_pandas(), train_data[target].to_pandas())
    lgb_val = lgb.Dataset(val_data[features].to_pandas(), val_data[target].to_pandas())

    # Define the parameters
    params = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": 0.01,
        "num_leaves": 31,
        "min_child_samples": 20,
        "max_depth": -1,
        "subsample_freq": 0,
        "bagging_seed": 0,
        "feature_fraction": 0.9,
        "bagging_fraction": 0.8,
        "reg_alpha": 0.1,
        "reg_lambda": 0.1,
    }

    # Train the model
    callbacks = [lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(period=20)]
    model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_val], callbacks=callbacks)

    # Save the model
    with open(f"model_lgb_{fold}.pkl", "wb") as f:
        pickle.dump(model_lgb, f)

    # Predict the validation data
    val_pred_lgb = model_lgb.predict(val_data[features].to_pandas(), num_iteration=model_lgb.best_iteration)

    # Evaluate the model
    score_lgb = np.sqrt(mean_squared_error(val_data[target].to_pandas(), val_pred_lgb))
    scores_lgb.append(score_lgb)

    print(f"RMSE for fold {fold}: {score_lgb}")

# Calculate the average score
average_score_lgb = np.mean(scores_lgb)

print(f"Average RMSE: {average_score_lgb}")

# Predict the test data and create the submission file
submission_df = pl.read_csv("../data/input/sample_submission.csv", try_parse_dates=True)
submission_df = submission_df.with_columns(pl.lit(0).alias("score"))

for fold in range(5):
    with open(f"model_lgb_{fold}.pkl", "rb") as f:
        model_lgb = pickle.load(f)
    test_pred_lgb = model_lgb.predict(test_merged[features].to_pandas(), num_iteration=model_lgb.best_iteration)
    submission_df = submission_df.with_columns((pl.col("score") + pl.Series(test_pred_lgb) / 5).alias("score"))

submission_df.write_csv("../data/output/submission_baseline+.csv")


/Users/marumarukun/Documents/compe/atmacup_15/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Training for fold 0...
Training until validation scores don't improve for 20 rounds
[20]	valid_0's rmse: 1.52485
[40]	valid_0's rmse: 1.50166
[60]	valid_0's rmse: 1.48519
[80]	valid_0's rmse: 1.4735
[100]	valid_0's rmse: 1.4643
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1.4643
RMSE for fold 0: 1.4642981825731645
Training for fold 1...
Training until validation scores don't improve for 20 rounds
[20]	valid_0's rmse: 1.54836
[40]	valid_0's rmse: 1.52446
[60]	valid_0's rmse: 1.50681
[80]	valid_0's rmse: 1.49395
[100]	valid_0's rmse: 1.48404
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1.48404
RMSE for fold 1: 1.4840401512940617
Training for fold 2...
Training until validation scores don't improve for 20 rounds
[20]	valid_0's rmse: 1.524
[40]	valid_0's rmse: 1.50073
[60]	valid_0's rmse: 1.48399
[80]	valid_0's rmse: 1.47169
[100]	valid_0's rmse: 1.46249
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1.46249
RMSE for 